In [1]:
# Train NER from a blank spacy model
import spacy

nlp=spacy.blank("en")

nlp.add_pipe(nlp.create_pipe('ner'))

nlp.begin_training()


In [2]:
# Import and load the spacy model
import spacy
nlp=spacy.load("en_core_web_sm") 

# Getting the ner component
ner=nlp.get_pipe('ner')

In [3]:
# New label to add
LABEL = "FOOD"

# Training examples in the required format
TRAIN_DATA =[ ("Pizza is a common fast food.", {"entities": [(0, 5, "FOOD")]}),
              ("Pasta is an italian recipe", {"entities": [(0, 5, "FOOD")]}),
              ("China's noodles are very famous", {"entities": [(8,14, "FOOD")]}),
              ("Shrimps are famous in China too", {"entities": [(0,7, "FOOD")]}),
              ("Lasagna is another classic of Italy", {"entities": [(0,7, "FOOD")]}),
              ("Sushi is extemely famous and expensive Japanese dish", {"entities": [(0,5, "FOOD")]}),
              ("Unagi is a famous seafood of Japan", {"entities": [(0,5, "FOOD")]}),
              ("Tempura , Soba are other famous dishes of Japan", {"entities": [(0,7, "FOOD")]}),
              ("Udon is a healthy type of noodles", {"entities": [(0,4, "ORG")]}),
              ("Chocolate soufflé is extremely famous french cuisine", {"entities": [(0,17, "FOOD")]}),
              ("Flamiche is french pastry", {"entities": [(0,8, "FOOD")]}),
              ("Burgers are the most commonly consumed fastfood", {"entities": [(0,7, "FOOD")]}),
              ("Burgers are the most commonly consumed fastfood", {"entities": [(0,7, "FOOD")]}),
              ("Frenchfries are considered too oily", {"entities": [(0,11, "FOOD")]})
           ]

In [4]:
# Add the new label to ner
ner.add_label(LABEL)

# Resume training
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [5]:
# Importing requirements
from spacy.util import minibatch, compounding
import random

# Begin training by disabling other pipeline components
with nlp.disable_pipes(*other_pipes) :

  sizes = compounding(1.0, 4.0, 1.001)
  # Training for 30 iterations     
  for itn in range(30):
    # shuffle examples before training
    random.shuffle(TRAIN_DATA)
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=sizes)
    # ictionary to store losses
    losses = {}
    for batch in batches:
      texts, annotations = zip(*batch)
      # Calling update() over the iteration
      nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
      print("Losses", losses)

Losses {'ner': 4.069681805959251}
Losses {'ner': 4.069684230851224}
Losses {'ner': 4.156802422963758}
Losses {'ner': 2.8984404841903597}
Losses {'ner': 3.865231498328285}
Losses {'ner': 6.205568944481701}
Losses {'ner': 1.837203322105779}
Losses {'ner': 2.8073690022918836}
Losses {'ner': 2.807956418664686}
Losses {'ner': 1.959394247453682}
Losses {'ner': 3.927128174098913}
Losses {'ner': 6.460989755912111}
Losses {'ner': 1.1042752687627626}
Losses {'ner': 1.1210485092076858}
Losses {'ner': 3.611098411463143}
Losses {'ner': 4.254951550121405}
Losses {'ner': 8.28694293859553}
Losses {'ner': 9.243206030787938}
Losses {'ner': 2.013493734621866}
Losses {'ner': 4.038300007165949}
Losses {'ner': 4.038360251153705}
Losses {'ner': 2.386202008826501}
Losses {'ner': 6.581934008010952}
Losses {'ner': 6.5856365860952675}
Losses {'ner': 3.998152563511326e-05}
Losses {'ner': 1.9953746470336604}
Losses {'ner': 1.9953954245832968}
Losses {'ner': 0.9484289026204493}
Losses {'ner': 3.6999550551630023}
Lo

In [6]:
# Testing the NER

test_text = "I ate Sushi yesterday. Maggi is a common fast food "
doc = nlp(test_text)
#print("Entities in '%s'" % test_text)
for ent in doc.ents:
  print(ent)

Entities in 'manufacturing unit for cleaning chemicals'
()


In [7]:
# Output directory
from pathlib import Path
output_dir=Path('/content/')

# Saving the model to the output directory
if not output_dir.exists():
  output_dir.mkdir()
nlp.meta['name'] = 'my_ner'  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

# Loading the model from the directory
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
assert nlp2.get_pipe("ner").move_names == move_names
doc2 = nlp2(' Dosa is an extremely famous south Indian dish')
for ent in doc2.ents:
  print(ent.label_, ent.text)

Saved model to /content
Loading from /content
